# Notebook for comparing the quality of block-assignment strategies

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.sparse as sp

from sbm.io import GraphLoader
from sbm.block_assigner import AssignerConstructor

In [2]:
g = GraphLoader.load(
    Path("/Users/lmmi/anon_sbm/data/processed/congress_twitter.edgelist"),
    force_undirected=True, # type: ignore
    )
#g = GraphLoader.load(
#    Path("/Users/lmmi/anon_sbm/data/processed/lastfm_asia.edgelist"),
#    force_undirected=True, # type: ignore
#    )

In [3]:
rng = np.random.default_rng(1) # type: ignore
k = 3

In [4]:
assigner_const = AssignerConstructor(rng=rng)
uniform_assigner = assigner_const.create_assigner(
    graph_data=g,
    min_block_size=k, # type: ignore
    init_method="uniform", # type: ignore
    )
prone_assigner = assigner_const.create_assigner(
    graph_data=g,
    min_block_size=k, # type: ignore
    init_method="prone_and_kmeans", # type: ignore
    )
metis_assigner = assigner_const.create_assigner(
    graph_data=g,
    min_block_size=k, # type: ignore
    init_method="metis", # type: ignore
    )
metis_ref_assigner = assigner_const.create_assigner(
    graph_data=g,
    min_block_size=k, # type: ignore
    init_method="metis_refine", # type: ignore
    )

In [10]:
uniform_blocks = uniform_assigner.compute_assignment()
#prone_blocks = prone_assigner.compute_assignment()
metis_blocks = metis_assigner.compute_assignment()
metis_ref_blocks = metis_ref_assigner.compute_assignment()

[158, 83]
[158]


In [11]:
print(set(uniform_blocks.block_sizes.values()))
#print(set(prone_blocks.block_sizes.values()))
print(set(metis_blocks.block_sizes.values()))
print(set(metis_ref_blocks.block_sizes.values()))

{3, 4}
{2, 3, 4, 5}
{3, 4}


In [12]:
# compute likelihood of the assignments
from sbm.likelihood import compute_global_bernoulli_ll_fast

uniform_nll = -compute_global_bernoulli_ll_fast(uniform_blocks)
#prone_ll = compute_global_bernoulli_ll_fast(prone_blocks)
metis_nll = -compute_global_bernoulli_ll_fast(metis_blocks)
metis_ref_nll = -compute_global_bernoulli_ll_fast(metis_ref_blocks)

print(f"""
Small graph likelihood comparison:
    Uniform assignment likelihood: {uniform_nll:.2f}
    Metis assignment likelihood: {metis_nll:.2f}
    Metis-refine assignment likelihood: {metis_ref_nll:.2f}
""")


Small graph likelihood comparison:
    Uniform assignment likelihood: 8261.40
    Metis assignment likelihood: 4820.68
    Metis-refine assignment likelihood: 4862.17

